In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import math
import torch
import sys
import json
import re
from tqdm import tqdm
from llama_attn_replace import replace_llama_attn

## Parameters

In [2]:
PROMPT_DICT = {
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
    "prompt_no_input_llama2": (
        "<s>[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} [/INST]"
    ),
    "prompt_llama2": "[INST]{instruction}[/INST]"
}

In [3]:
model_name = "Yukang/LongAlpaca-7B"
cache_dir = "../../cache"
context_size = 32768

## Config, Model and Tokenizer

In [4]:
replace_llama_attn(inference=True)

In [5]:
config = AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)

In [6]:
orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len and context_size > orig_ctx_len:
    scaling_factor = float(math.ceil(context_size / orig_ctx_len))
    config.rope_scaling = {"type": "linear", "factor": scaling_factor}

In [7]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        config=config,
        cache_dir=cache_dir,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model.resize_token_embeddings(32001)

Embedding(32001, 4096, padding_idx=0)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        model_max_length=context_size if context_size > orig_ctx_len else orig_ctx_len,
        padding_side="right",
        use_fast=False,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Inference: MB

In [10]:
def build_generator(
    model, tokenizer, temperature=0.6, top_p=0.9, max_gen_len=512, use_cache=True
):
    def response(prompt):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        #streamer = TextStreamer(tokenizer)
        
        output = model.generate(
            **inputs,
            max_new_tokens=max_gen_len,
            temperature=temperature,
            top_p=top_p,
            use_cache=use_cache,
            #streamer=streamer,
        )
        
        out = tokenizer.decode(output[0], skip_special_tokens=True)

        out = out.split(prompt.lstrip("<s>"))[1].strip()
        return out

    return response

In [11]:
model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

respond = build_generator(model, tokenizer, max_gen_len=1024)

In [12]:
prompt_no_input = PROMPT_DICT["prompt_llama2"]

In [13]:
question = "Summarize the meeting transcrip. Summarize each item in two sentences."

### Single Example

In [14]:
material = "Item 21-1322: speaker 0: All right. We're going to go ahead and get started this evening. We're coming back from recess here. And this evening we have one public hearing tonight for those participating in person when called upon. We ask that you please come to the podium and on the presentation monitor on the wall you will see your time counting down for\nspeaker 0: those participating virtually when called upon. Please wait until our meeting hosts promote you to speaker. When you are promoted, your screen will flash and say Reconnecting to meeting. Please do not leave the meeting. You will be reconnected and will need to turn on your camera if you have one.\nspeaker 0: And your microphone, you'll see your time counting down at the bottom of your screen. Once you've finished speaking, you will change back to participant mode and see your screen flash one more time. All speakers should begin their remarks by telling the council their names and cities of residence and if they feel comfortable doing so.\nspeaker 0: Their Home Address. If you've signed up to answer questions only, please state your name and note that you are available for questions of counsel. Speakers will have 3 minutes. There is no yielding of time. If translation is needed, you will be given an additional 3 minutes for your comments to be interpreted.\nspeaker 0: Will alternate between in-person and virtual for efficiency by calling in-person participants and then alternating to virtual participants. Speakers must stay on the topic of the hearing and must direct their comments to the council members. Please refrain from profane or obscene speech.\nspeaker 0: Direct your comments to council as a whole and please refrain from individual or personal attacks. Councilmember Sandoval, will you please put Council Bill one, three, two, two on the floor for final passage?\nspeaker 5: Yes. I move that bill. One, three, two. To be placed on. Very final passage.\nspeaker 0: Sorry. All right, we've got it on final consideration for passage. It has been moved and seconded the required public hearing for council bill one, three, two, two is open. And we're going to have the staff report here from Councilmembers Flynn and Ortega.\nspeaker 6: Thank you, Madam President. Councilwoman Ortega and I are very pleased to bring this forward for the last couple of years. Some of us have been discussing how best to recognize the contributions of Kathy Reynolds, Councilwoman Kathy Reynolds to this body, to this city, and to all of its people.\nspeaker 6: And I think they are only regret is that she is not here with us any longer to see this happen. But I don't know if Councilwoman, if you want to make a few opening remarks.\nspeaker 0: I just want to thank Councilman Flynn for working together in bringing this forward and reaching out to former colleagues, many of which are either here or online. That will be speaking to you shortly. But when we explored what made sense, naming the chambers was like the perfect solution to honor Cathy Reynolds.\nspeaker 0: And it will be something that will, you know, last forever. I mean, this this building was built to last forever. And Cathy's contributions were just enormous. And we'll we'll walk through those shortly.\nspeaker 6: Thank you. Before we get started, I do want to recognize one person here in the chamber that we haven't seen in a very long time. There are as many former members of this body who are here. But there's one gentleman here who served on the third floor for many years, and that is our former city attorney, Dan\nspeaker 6: Muse, who is here with his wife, a SEAL Harmer. Dan, thank you for your service to the city. Dan was city attorney for many years under Mayor Wellington Webb and I spoke with him right before this. He said Cathy is just one of a kind and he's he's very pleased that council's taking this step.\nspeaker 6: Let me tell you exactly why we have filed this bill and believe that this is something that ought to be approved unanimously. Up until 1975. No woman had ever been elected to this body. In the year 1975, two women were.\nspeaker 6: Kathy Reynolds holds the distinction of having been elected first because the second woman elected that year. Kathy Donahue had a runoff. And so Kathy Reynolds was elected in May and Kathy Donahue was elected in a runoff in June in District ten, which I don't think was perfect at that time.\nspeaker 6: Councilman Hines But but she helped to make it so and so. Two women took office that year sitting among 11 men. She was reelected a total of seven times. There are three other members of this body. Who have been elected seven times.\nspeaker 6: One was the longtime councilman from what was then District eight and five points, and that's Elvin Caldwell. Councilman Caldwell, though, did not serve the entire 28 years. So Councilwoman Reynolds has the record of tenure. That is the longest.\nspeaker 6: Councilman Caldwell was appointed during his seventh term as manager of safety by Mayor McNichols, so he resigned his seat during that seventh term. The other council member who served seven consecutive continuous terms is the gentleman memorialized on the plaque by the door into our office.\nspeaker 6: And that's Paul Hensel, who served seven terms out of District four in southeast Denver. The other member of this council, who will have served seven complete terms, is standing to my left, and that is our councilwoman, Debbie Ortega.\nspeaker 6: Councilwoman Ortega, Ortega's tenure is discontinuous. She served four terms until term limits took effect in 2003. And then, as we all know, she was reelected to this body in 2011 and is serving right now her seventh term and will be termed out at the end of this particular service.\nspeaker 6: I mean, I think it is remarkable and very honorable of Councilwoman Ortega that while she will match Councilwoman Reynolds in years of service, she is a co-sponsor to this and is is fully supportive. I don't know, Councilwoman, if you want to make some remarks on that.\nspeaker 0: So I just want to say I had the pleasure of serving with Councilwoman Reynolds. Alongside many of her colleagues who are here tonight. And she was an incredible individual who was a mentor to all of us, newbies who came on.\nspeaker 0: When I got elected, there were four new council members and it was enough to sort of shake things up a little bit in terms of looking at doing things a little bit differently. But she truly was an individual who brought a lot of knowledge and was always willing to sit down and answer questions and just offer her\nspeaker 0: wisdom. When I was struggling with with varying issues, we worked very closely together on a build out of the Central Valley. The Pepsi Center was in my district. She and I served on the committee together. And, you know, we'll talk more about her contributions.\nspeaker 0: But it it is just an honor to be able to do this tonight and to recognize Cathy in this way for her many, many contributions to this city that will be in place for many, many years into the future.\nspeaker 6: She, Kathy Reynolds will have the longest continuous service of any council member by a quirk of a charter. And as former city attorney muse may remember this, but after the 91 mayoral election and the very hectic runoff that year, that led to Mayor Webb taking office almost immediately after the runoff and not having the time that should\nspeaker 6: be afforded to put together an administration. There was a charter amendment put on the ballot to push the inauguration back to the third Tuesday in I'm sorry, the third Monday in July. It used to be the first Monday in July.\nspeaker 6: So Councilwoman Reynolds will have that record over Councilwoman Hensel on the wall over there, excuse me, by two weeks. During her tenure, Councilwoman Reynolds served alongside.\nspeaker 9: 38.\nspeaker 6: Different members of the council four times. She was elected council president. She actually served five terms because when Councilman Caldwell was resigned to become manager of safety. Councilwoman Reynolds had been pro tem and she became acting well. She became the president for the rest of Councilman Caldwell's year as president.\nspeaker 6: Eight times she was elected as council president pro tem. Since the election of 1975, when council members Reynolds and Donohue became the first two women that seemed to open the floodgates for women in office. Here's one of the flood gates right here.\nspeaker 6: And we have in seated in the audience, we have a number of women who were elected to this body subsequent to that. And I don't know if this was the appropriate time. Could all members. It is because I just said it could.\nspeaker 6: All members who have all persons who have served on this body, please stand up and be recognized. Thank you. It should be noted, though, that Kathy Reynolds was not the first woman to serve on the body. I purposely put in the presentation first woman elected because of one of those vagaries of history.\nspeaker 6: In 1935, as you can see in this newspaper article, who was there was Eugene Bartholdi from North Denver, who resigned.\nspeaker 1: About.\nspeaker 6: Four months before the end of his term in 1935. And Mayor George but goal appointed a woman, Elissa Paladino from North Denver to fill out the last four months of that term. And she did not run again. And I wanted to draw your attention to her quote at the bottom of that story.\nspeaker 6: It says, You can bet Mrs. Paladino said, you can bet I won't neglect my cooking and sewing. I cannot imagine don't stab me in the back here. I cannot imagine those words being said to a newspaper reporter by Sal Carpio or Ted Hackworth or Paul Schwalm or Tim Sandoz.\nspeaker 6: Right. Or who else is your Charlie Brown? Charlie Brown here now. So since that election go from 1979 onward, 31 of the 56 people who've been elected to this body since 1979 were women. A majority, 31 out of the 56 people elected to this body were women.\nspeaker 0: This is crazy. I also want to acknowledge the men who are not here, who are no longer here, who Cathy served with. And that was Councilman Bill Roberts, Sal Carpio, Hiawatha Davis. Bill Zeigler. Elvin Caldwell. Sam Sanders. Ted Hackworth.\nspeaker 0: Bob Kreider. Paul Hensel. Ed Burke. Joe Nolan. Larry Perry and Ken Macintosh. These are all you know, this was when it was predominantly a male body. But these are men who are no longer alive. And I wanted to acknowledge them.\nspeaker 6: Thank you. So I want to go on to some of the policy initiatives that we believe merit naming this room where these events took place, where these initiatives were passed and the debates took place in recognition of the person who advanced them or primarily advanced them, because I think Councilwoman Reynolds would not ever say that she did\nspeaker 6: this alone. She was collaborative. She worked with anybody and everybody who could get the ball across the goal line. Right. So one of the one of the nation's first gay rights laws passed in Denver in 1990 with significant debate and controversy, such controversy that led in part because other cities around the state also passed similar laws around\nspeaker 6: the same time. But it resulted in an event many of us remember it came out of Colorado Springs was Amendment two in 1992, and that was a constitutional amendment that passed by the people of this state that said no city can adopt any gay essentially any gay rights laws.\nspeaker 6: This was challenged up to the state Supreme Court and eventually went to the United States Supreme Court in 1996 and resulted in the landmark ruling that extended equal protection based on sexual orientation. You will hear tonight. You will hear tonight from two individuals who were central to that debate.\nspeaker 6: They. One of them flew in from Portland, Oregon, to be here tonight to talk to you about those times and some of the hate and division that. Roll out of that debate. Councilwoman Reynolds also was a leader in changing the city's most restrictive zone district regulation r zero zoning under the old 1956 code.\nspeaker 6: R zero zoning was the most exclusive in the city. And one of the regulations was that. No unrelated adults could live together. Unmarried couples could not live together. Effectively, this was a barrier, a bar prohibition. Two gay couples living in some of our finest neighborhoods.\nspeaker 6: You know, it only applied in the zero zone, but we're saying, well, you all can live somewhere else, but you can't live here. And she was instrumental in leading the charge to have that repealed. She also led a successful drive for the budget amendment that we know today as the the mill levy for people with intellectual developmental\nspeaker 6: disabilities. That was hotly contested both here on the floor and on the third floor. This was during the Pena administration, I believe, excuse me. And there was actually kind of a raucous debate. I won't name the names. And I think who told me about this?\nspeaker 6: I think it was former councilman Ed Thomas told me about this. And Councilman Ortega, I think you were council president that year that this was being debated. Councilwoman Reynolds got into a heated argument on this floor with a male member of the body who shall remain nameless.\nspeaker 6: And it got so intense that I was told the Council on President Ortega said from that desk, Madam President, would you to take it out in the hall? Whereupon Councilman Thomas shouted from his desk. My money's on Reynolds.\nspeaker 6: And. And he won because that mill levee did pass. And it has been extended by the voters of Denver once Tabor came into effect. The voters of Denver approved increasing that mill levy sometime in the early to late nineties.\nspeaker 6: I don't have the exact year.\nspeaker 0: Let me just add, when when that passed, it was only generating like around $1,000,000. That fund now generates upwards of it's a full mill. I don't remember what that is now, but when I came on in 2011, it was at about 13 million, and now we think it's closer to 20 million.\nspeaker 0: So it's making huge impact on people with developmental disabilities. And Cathy should be so proud of that work that she did.\nspeaker 6: And finally, the other initiative that we want to bring to your attention is one that I recall vividly when I was when I was a reporter, the Rocky Mountain News. I remember the intense level of threats that threats that came in to this office over here against the councilwoman because she was the prime sponsor of the first\nspeaker 6: assault weapons ban in Denver, the one that eventually survived court challenges and challenges from the state legislature to preempt our home rule ability to take such action in the here in the city. I remember many times after Monday night that she had to have a police escort to her home in Park Hill because of the threats that\nspeaker 6: were coming in.\nspeaker 0: And we have had the pleasure of the sheriff's department being at our Monday night meetings on a regular basis. And I can remember having to be escorted out to our cars after that legislation passed, because the threat wasn't just directed at Kathy, it was at the whole body because we passed it.\nspeaker 6: Some things don't change, do they? Project initiatives. I intentionally led this presentation, Madam President, with policy initiatives. But I also want to talk about project initiatives because Councilwoman Reynolds, along with many of the folks you see here behind us.\nspeaker 6: Played central roles in the development of so many things that the people of Denver right now take for granted the Colorado Convention Center, the new Colorado Convention Center. Where should it go? Remember, comes from there were three different sites that were Central Platte Valley, the Golden Triangle, or right downtown.\nspeaker 6: Imagine what this city would look like if instead of a transit hub behind Union Station, there was a convention center. I think that was the Marvin Davis site. And then the site behind where the library is now. There wouldn't have been a library expansion if we had made that mistake.\nspeaker 6: The Convention Center Hotel, which is now the Hyatt Regency Denver International Airport, of course, Coors Field, the new Broncos stadium, the Pepsi Center, which is now changed its name, I think, to Ball Arena. I used this picture. Thank you to Leah and Carla SHAPIRO from The Denver Post, the editor who allowed us to use these photos.\nspeaker 6: This is. This is a photograph that really says something about Kathy Reynolds being working with a group of men right over there in that corner that if you recognize that's that window over there, she is the person with her.\nspeaker 6: Back to you and her purse on the floor. I'm going to steal one of Rick's stories. Rick told me that one day a constituent got in Cathy's face and said, You know what your problem is, Reynolds? You never saw a project that you didn't like.\nspeaker 6: And if there's anything you can say about Kathy Reynolds, one of the first things will be she was very quick witted. She would think of the rest of the responses. Take me three days to come up with. Right?\nspeaker 6: She got right back in the guy's face and said, no, you never saw a project that I didn't like. She was also a leader in representing Denver outside of this room. She served all 28 years on this body as the rep, as one of the representatives to the Urban Drainage and Flood Control District Board.\nspeaker 6: What we now call the Mile High Flood District, including the last 23 years as the board chair. And you will hear during the public hearing from the former head of the of what they called advocate, I remember a councilwoman around was giving reports the committee saying she had gone to an urban drainage luncheon that month and giving\nspeaker 6: a report on. I thought nothing is less appetizing than an urban drainage luncheon. I have no idea what they served there, but doesn't sound appetizing. She was twice elected as president of the Colorado Municipal League, and in your email you have a statement of support from the former president of the Colorado Municipal League, Sam Mamet, who is\nspeaker 6: on travel and could not be here tonight and deeply regrets that he could not be here. But he sent a statement that is in your email. She was elected president of the National League of Cities in 1987, serving as basically the chief spokesperson for municipal issues in the country, testifying in Congress several times during that year as\nspeaker 6: president. And after leaving council, she was a charter member of the Citizen Oversight Board and served on the Colorado Convention Center Hotel Board.\nspeaker 0: I want to just. Yes. So when she was the president of the National League of Cities, she encouraged all of us to get involved with the National League of Cities. We attended many of their annual conferences. It gave us an opportunity to learn about best practices being done in other cities, to come back with new and different\nspeaker 0: ideas. And I just appreciated that about her because it just continued to help with the growth and the knowledge and the ability to just look at things differently.\nspeaker 6: Thank you. I love this picture. It shows a bit of determination and kind of the pursed lips. Right. And she must be really mad at something. I don't know what it was. Maybe it's one of my stories in The Rocky.\nspeaker 6: I don't know, but. When we were looking a couple of years ago for where could we find a place to recognize Cathy's contributions? We looked around, but she served at large, and so she didn't have a district where there was a park that she championed or a rec center like the Kala Madison Rec Center or the Shimer\nspeaker 6: Rec Center or the Hiawatha Davis Recreation Center. She was citywide and we were kind of brainstorming. There's no park, there's no rec center. And it was one of our former legislative analysts. You might remember Shelley Smith. Who just popped up and said, Why not the chamber?\nspeaker 6: Because of the variety, the breadth, the depth of her accomplishments. What better place than the room in which they occurred? The room where it happened, if you will. And so that's why we bring this this bill forward. Let me before you start the public hearing, Madam President, one former member who cannot be here or is not here\nspeaker 6: is watching online. That is Councilwoman Rosemary Rodriguez from Council District three and who served in several other roles in city government. She sent me a message that she I don't know if she wanted me to read it. Maybe she'll kill me when she when I see her next.\nspeaker 6: But here's what she said. Here's my synopsis of Kathy Reynolds. There is a song from the show Gypsy, that is sung by Mama Rose. The relevant lyric is this. You either got it or you don't. And guys, I got it.\nspeaker 6: That describes Cathy's relationship with Denver. She knew the code. She knew the players. She knew the people. What a wonderful idea to name the meeting room for the woman who convened countless meetings for Denver's diverse, disparate and disagreeable.\nspeaker 6: She knew our people and loved us all.\nspeaker 1: Full stop.\nspeaker 6: I also want to mention I don't I didn't see I don't know if we have online. Former Councilwoman Elena Wedgeworth. She online is she signed up to speak as well? Excellent. Okay. Thank you. And so, Madam President, with one more moment of indulgence, I want to mention one other person who is here who flew in from Washington\nspeaker 6: , DC. Where? Oh, Judy. Judy Gold was Kathy Reynolds council aide and started at a time when it was very unusual for us to have assistants working with us. Right. So, Judy, I want to thank you for coming in, flying in from D.C. to be here for this special moment.\nspeaker 6: And with that, Madam President, I'm told Mayor Webb is also on line. To speak since Mayor Webb is on line. I wanted to show everybody that among her effects, 1996, 1997, the mayor's box at Mile High Stadium. The tickets were always divvied up between the mayor's office and the city council.\nspeaker 6: And there was quite a blowup. Maybe Mayor Webb. He might not address this. So let me do it.\nspeaker 1: These are the last four.\nspeaker 6: Season ticket packages from the 1997 season for the mayor's box and nine in 1997 that never got distributed. And there are still ink in Cathy's files. And so it's a little bit of history. I don't know if it's too late to get them to the mayor now, but I'll give them back to Rick.\nspeaker 6: With that, Madam President, I don't know if, Councilwoman, you have any more remarks before we open the hearing.\nspeaker 0: The only thing I would add is that when. Judy Gold was a council aide to Councilwoman Reynolds. Ramona martinez was one of the other aides. I think they were the first two that were hired on to work for members of city council.\nspeaker 0: And then Ramona went on to replace Sam Sanders as a member of council. But they paved the way for council as well. And back then, there was only one aide for council member. It grew. It has grown over time.\nspeaker 0: So thank you all for allowing us to do this presentation. We're truly honored to bring this forward and look forward to hearing from her former colleagues as well. Very good. Well, thank you. Councilmembers Ortega and Flynn for the presentation.\nspeaker 0: We always appreciate learning a bit more about the history and the remarkable accomplishments of Kathy Reynolds. This evening, we have 14 individuals signed up to speak and we're going to call the first two who are joining us online.\nspeaker 0: And then the rest of the speakers are here in chambers with us. And so our first speaker this evening is Wellington Webb. And we need to see if you are on line. We have one call in number and we believe that that is.\nspeaker 0: Former Mayor Webb. But we're not. Seen it now. Okay. All right. We're going to go ahead and we will try to get him back into the queue. But we're going to go ahead and go with Elba. Wedgeworth is our next speaker.\nspeaker 0: Okay. We can. We want you to go ahead and we can hear you now. Go ahead, please. We're not able to hear you. You might have to unmute. I think we're having some problems with the mike. And so we might need to have you leave the Zoom meeting and log back in.\nspeaker 0: And when you log back in, make sure to connect your audio up. We're showing that your audio's not connected. And so we'll go ahead and try to get those two online speakers back into the queue. And so in the interest of time, we'll go ahead and transition into chambers.\nspeaker 0: And the first speaker in chambers is Tim Santos.\nspeaker 9: Good evening, Madam President. Members of Council. My name is Tim Sandoz and I currently reside in Green Valley Ranch and am proud to be able to be here tonight to speak to you about my dear friend and mentor Cathy Reynolds.\nspeaker 9: Now, Cathy Reynolds and my father, Sam Santos, were elected at the same time, in the same term, and they came in to serve together. At that time, my student council advisor was Mr. Rick Reynolds, who just happened to be Cathy's husband.\nspeaker 9: He is the one who taught me Robert's Rules of Order. He taught me Robert's Rules of Order. Kathy taught me how to use those rules, and she taught me very, very well. Kathy was one of the smartest people you ever come across, and she was always very witty, also knew how to count votes.\nspeaker 9: So in addition to taking me under her wing, as she did because my father had passed away and as dad, I have to mention so many other women that were on the council. Debbie Ortega, Ramona martinez. Happy Haines.\nspeaker 9: They all really embraced me because I was Sam son. Sam had passed away and so they were helping me to understand the rules, how we worked, what we did. Well, these same women helped me with the very first major piece of legislation I was passing, and I was sitting right where Councilwoman Sandoval is sitting at the time\nspeaker 9: . And I was waxing eloquent about how well we had done all the work that had happened. And then slowly a message from Councilwoman Reynolds, who happened to be present at that time, skipped across table to table to table till it got to me.\nspeaker 9: And I opened it up and it said, You have seven votes. Keep on talking. You'll only have six. That was Kathy Runnels. She was sharp. She was quick. She was very and she also was very straight forward about the things that I needed to do and helped really for me to grow as a member of council.\nspeaker 9: Now, as I look at the time. Oh, it's not from Kathy Reynolds, she said. You're talking too much. Thank you all very much.\nspeaker 0: All right. Thank you very much. Oh, we're going to go ahead and move back online because I believe we've got Elvira Wedgeworth. We've got her back in the queue. Yes.\nspeaker 9: Can you hear me now? Hello? Can you hear me?\nspeaker 0: Yes.\nspeaker 9: Okay. Thank you. And good evening, Madam President, and members of the Denver City Council. My name is Alper Wedgeworth and I live in the Whittier neighborhood in Denver and District nine. I also wish to thank and acknowledge Councilwoman Glenn and Councilwoman Ortega for their efforts to name the Denver City Council Chambers in honor of Cathy Reynolds.\nspeaker 9: I work with and personally knew Cathy Reynolds since 1989 when I began my public service career serving as a senior analyst on the Denver City Council staff. I worked directly with Kathy on some of her legislative policy issues regarding the ban of assault weapons in the city of Denver and many others.\nspeaker 9: I was always impressed, very impressed with Kathy's thoughtfulness, her tenacity and her sense of humor. She was also always very kind and encouraging and a great steward of this institution. Kathy always encouraged me to learn and have a self-education on public policy in a way that would be balanced and also, to be fair, to approach all issues\nspeaker 9: . Kathy was a very strong woman who was able to break down the stereotypes of leadership during that time. As was said tonight, very few women have this leadership role in government and to break those glass ceilings so other women would be able to follow.\nspeaker 9: She was very determined. And at times quiet and always thoughtful and always grounded in her leadership and her ability to make a difference and to communicate and have to say again. She had a tremendous sense of humor. Kathy would always make you laugh about a lot of different things all the time.\nspeaker 9: I truly appreciate also her courage and her confidence and her leadership and her passion. Our public service as she spoke, to be open for change and to be positive and proactive, suggesting things to you that you should consider.\nspeaker 9: I will always be thankful to Kathy for her encouragement and her friendship and the many things I learned from her over the years. When I became the Denver City Council president myself. Kathy was many great things and woman of many accomplishments.\nspeaker 9: Ever. I believe that she was proudest of being a wife, a mother, grandmother, and a friend mentor to everyone. Now and in the future. I will always think fondly of Kathy and the role she played in my journey of public service and her leadership on behalf of the city of Denver.\nspeaker 9: I will always, always be thankful to her. To Rick and the family. We were blessed to have her in our lives, and we thank you for sharing her with us. So thank you, Cathy, and thank you, members of Council for your consideration.\nspeaker 9: Thank you.\nspeaker 0: Our next speaker is in chambers, Stephanie Foote. Good evening. Members of Council, um, councilman flynn and Councilwoman Ortega. You guys did just an incredible job that brings tears to my eyes to see these pictures of Cathy. Because I remember I was on council.\nspeaker 0: I was elected in 1983 and I served three terms. I was a really a newbie to government, but I knew that in 1983 I had a nine month old baby and Mike, my campaign, when I talked to people, I said, if we don't do something right now, Denver's not going to be here when my kids grow up\nspeaker 0: and there's not a place where they can work. I came to council. I had no idea how this process worked. I was I know you're going to find this hard to believe, but I was terrified. And I had to learn the rules.\nspeaker 0: I had to watch what everybody else was doing. Cathy mentored me through all that, and then at the end of the first year, she took me in her office and as only Cathy could, she said, Steph, you've been here a year now.\nspeaker 0: You can talk from that time on, whenever I needed any kind of guidance, I needed help. I needed to understand from the absolutely best public policy mind that I have ever known or been close to. She helped us all.\nspeaker 0: All of us. I think that had the the beauty of spending time with her to gain that knowledge that we did not come here with. We dealt with huge issues. We you saw her list. We came in at the time.\nspeaker 0: I came in with the new mayor, Mayor Federico Pena. And it was really a struggle to form coalitions, to understand what everybody's point of view was. But you all know that, and Cathy in particular, that we had to have all the arrows pointing in the same direction if we were going to do the right thing for Denver\nspeaker 0: . So thank you all so much for this. This is just an incredible tribute. It couldn't go to someone more deserving. And Cathy, we love you and we miss you. Thank you. Our next speaker is Happy Hands.\nspeaker 9: Good evening, Madam President and members of Council. My name is Happy Haines. I live in North Park Hill. Actually, I live mostly across the street these days as you live here. Thank you, Councilman Flynn and Councilwoman Ortega, for this brilliant idea.\nspeaker 9: I can't think of a more fitting tribute to Cathy Reynolds. A real state's person and policy leader unparalleled. And hopefully it'll inspire a lot more wonderful decisions for many, many decades to come. Um, uh, I. I knew Cathy Reynolds the year she was elected was your first.\nspeaker 9: The councilman that I served under as a council aide, served in the first, was elected that same year when that district was first created in northeast Denver around some concerns about equity in this city. And so I got to know Cathy Reynolds from on the perspective of a city council aide.\nspeaker 9: So when I was elected, as many people here have said, Cathy was a mentor and she was more than that. She was a friend. But when I was first elected, I got called into the room to Stephanie and she told me that I would be consider myself proud to follow my predecessor in every respect but one, and\nspeaker 9: that was to keep my speeches much briefer. And I tried to follow that advice most often. Um, but I think as was mentioned also, she was a teacher. I know Rick really was the teacher in the family, but Cathy was too.\nspeaker 9: And she considered a government governance and public policy a lifelong learning experience. And so you've heard from the presentation all of the organizations that she belonged to, both locally and in this state and across this country. And she used those opportunities and encouraged all of us, likewise, to use those as learning experiences, because we didn't know at\nspeaker 9: all here. She didn't know it all, none of us did. And so we could take advantage of what was happening across the state, across this country and constantly learn. And that's what she was in every organization. She was such an extraordinary leader.\nspeaker 9: And Cathy was a kind of leader. She was a go to person in a get it done kind of person. So she believed in getting things done, not at the expense of principle, but she believed that there were always there was always a way, not only one way, but a way.\nspeaker 9: And so she used her power of creating relationships and listening to people and connecting with people both inside this chamber and outside throughout the city to bring people together to hash it out. I never saw her run away from a fight, ever.\nspeaker 0: But we're at our time. And so.\nspeaker 9: I.\nspeaker 0: Want to let you know.\nspeaker 9: So thank you very much for allowing us.\nspeaker 0: Thank you. Thank you. Next, the good guys, Scott, Scott Tucker. Scott Tucker. There you go.\nspeaker 6: Thank you, Councilmember. My name is.\nspeaker 3: Scott Tucker, and I was the.\nspeaker 6: Executive director of the Urban Drainage and Flood Control District. And Kathy was my boss for 23 years. She was chairman of the board of the District. Kathy had a wonderful sense of humor. And I think you see something is kind of common as we go through this.\nspeaker 6: But as she always referred to the UDV seed as a deferred, you'd f you D and I was always worried when she made presentations before like the in the Senate or the House and legislative issues. And I was always grateful when she did not use that term.\nspeaker 6: You might remember the seventies and eighties were really.\nspeaker 1: Very.\nspeaker 6: Tumultuous times between the suburbs and in the city. Some of you might see some of the seasons, cities, season citizens might remember the POUNDSTONE amendment that restricted Denver's ability to annex. There were just a lot of difficulties there.\nspeaker 6: The board of directors was made up of the mayor of Denver, three Denver council people, four mayors appointed by the governor from suburban communities and.\nspeaker 1: Five county commissioners, and then two.\nspeaker 6: Engineers appointed by the board. And she was able to handle them very, very nicely. And out of that mixture were 25 years of of getting things done. And that's, I thought was just really, really fantastic. She was very smart and intellectually, both intellectually and politically, and she had a wonderful way with people and of expressing yourself.\nspeaker 6: And quite simply, people trusted Kathy and believed her. The highlight of the month for me when I was I meet with Kathy for lunch right before the board meeting two or three days before the board meeting, and that went on for 23 years.\nspeaker 6: They added it up that it was not like 240 or 230 lunch meetings.\nspeaker 1: We discuss the issues of drainage and.\nspeaker 6: Flood control and all those arcane subjects. And finally, I'd like to thank Kathy and Rick. They introduced my family to Gilbert and Sullivan. And Rick, thank you very much for that refrigerator that and thank you very much for allowed to speak.\nspeaker 0: Thank you. Our next speaker is Tony Ogden.\nspeaker 9: Good evening, Madam President. Good evening, members of the Council. My name is Tony Ogden and I, my husband and I currently reside in Portland, Oregon. I was a Denver resident from the early eighties to 2002, and I had the pleasure of working with candidate Kathy Reynolds on some key initiatives at that time.\nspeaker 9: It is a privilege to be here to speak in support of this bill tonight, and my thanks to her for bringing it forward. You're going to hear lots of stories about Cathy. You've heard a lot of stories about Cathy.\nspeaker 9: Her storied career precedes us. I will speak to one issue in particular. I lasted and testified before this council in 1990, when only one of you was a member of this council. Many of you also were here in support of Denver's Comprehensive Anti-Discrimination Bill Ordinance Number 623.\nspeaker 9: I worked alongside several advocates in developing the original draft of that bill and helped found the Equal Protection Ordinance Coalition with the draft bill in hand. T shook and I knocked on Cathy Reynolds door in about 1988. Cathy opened the door wide for us and offered her support and sponsorship.\nspeaker 9: There seems to be a no brainer today, but in the eighties and the nineties, rights for the LGBTQ community were still being hard to find. Few jurisdictions in the country had a gay rights bill. Only two other cities, Aspen and Boulder, in the state of Colorado, had an ordinance ordinance that included sexual orientation.\nspeaker 9: When we first approached Cathy, she was working to an amendment amend Denver's ah zero zoning. And so we worked with Cathy and others to change the zero zoning because it would serve as the basically the foundation for us to be able to do the work on the Civil Rights Ordinance.\nspeaker 9: Cathy was keen on political strategy, understood the importance of coalition building and of patients. Cathy did not shy away from and what was at times a very ugly and hateful opposition during that time. She was steadfast in her support through an attempted repeal of only the sexual orientation portion of the ordinance.\nspeaker 9: The fight over amendment to and she celebrated with us vindication when the Supreme Court did decide Romer versus Evans. Kathy opened her door when many others would not. On this and so many other issues Kathy supported. She demonstrated leadership and is in the face of significant obstacles, not out of political gain, but out of true conviction, because\nspeaker 9: it was the right thing to do for the people of the city and county of Denver. As others have and will continue to say about Kathy. She loved this city. She was dedicated to it and committed to making it a better place.\nspeaker 9: I would go so far to say, as Cathy's leadership and courage in fighting discrimination is one reason Colorado has an openly gay governor today. I am proud to have worked with Kathy and to have regarded her a friend.\nspeaker 9: Renaming Council Chambers in her honor speaks to what it means to be a public servant. Leadership. Dedication. Commitment. Honor. Integrity. Her name on this chamber will serve as a constant reminder of those values. I appreciate the opportunity to be here before you tonight and to urge your support of this bill.\nspeaker 9: Thank you very much.\nspeaker 0: Thank you. Our next speaker this evening is Rick Reynolds.\nspeaker 9: Oh, boy. Where? My glasses.\nspeaker 1: Really?\nspeaker 9: Sorry. First of all, I'd like to introduce my son, Matt. My son Bob. Kathy's boys. Thank you all for being here. My name is Rick Reynolds. I live in I am a Denver native and I live at 16th and Locust in Denver.\nspeaker 9: How can anyone talk about three decades of service in only 3 minutes? I won't even try. But what was Denver like? Before Kathy Reynolds served? There were no women in city council. There were parts of Denver where unmarried couples could not live together and same sex marriage was illegal.\nspeaker 9: Those couples did not enjoy the same rights as heterosexual people did. There was no convention center or convention center hotel. There was no city lobbyist at the state capital. The council members did not have a staff. Kathy how did the first one?\nspeaker 9: Judy Gold. Neighborhood organizations were not and were not notified of zoning changes and liquor permits. One could legally own an assault weapon in Denver. There was no dedicated, dedicated funding for the developmentally disabled. There was no citizens oversight board or independent monitor.\nspeaker 9: There was no cable TV in Denver. There were no Colorado Rockies or Colorado Avalanche. Coors Field did not exist. The 16th mall, the 16th Street Mall did not exist. The early Caulkins Opera House was in. It was a small arena for conventions and basketball games.\nspeaker 9: The Better Concert Hall did not exist. Colorado Open Journey did not exist. The Bonfils Theater complex did not exist. DEA did not exist. Lowry was an Air Force base. Kathy had a huge hand in all of these changes.\nspeaker 9: With these changes have happened without her. Maybe so over time. But she pushed them. In her own words. Kathy, if you ever wish to be a celebrity. What would you like to be famous for?\nspeaker 3: I've had about all the celebrity I can handle from one time or another. What I'd like to be known for would be kindness, generosity.\nspeaker 9: And good humor. I'll never know.\nspeaker 0: Thank you. Thank you. Our next speaker is Ramona martinez. Well, good evening, members of city council.\nspeaker 9: Madam President, it's this very difficult for a lot of us tonight to get up here and express how we feel about Cathy Reynolds. I met Cathy before she got elected to city council. We went to Washington, D.C. in 1974, a long time ago, and a women's conference that was for women in leadership to learn how to become\nspeaker 9: leaders and leadership roles. Well, the city.\nspeaker 0: Colorado's.\nspeaker 9: Delegation is where I met Cathy because.\nspeaker 0: We had lunch and dinner.\nspeaker 9: Together during the week. We were out there in D.C.. And Cathy got elected right after that conference at Washington. A couple I don't know if it was a year or two later, but she got elected. I grew up in this city.\nspeaker 9: I grew up in downtown Denver. I'm back in my city.\nspeaker 0: In downtown Denver.\nspeaker 9: I lived here all my life. There were never any male, any women on the city council when I was growing up. There were never. Except for album Colwell. I think he was the first minority to ever sit on this city council when I was growing up.\nspeaker 9: And Kathy, she broke the glass ceiling for a lot of you up there. She broke it for me. Look at this.\nspeaker 0: This council is now.\nspeaker 9: Made up of the diversity that this city is all about. Not only did she help build and help us negotiate the Mile High Stadium, I sat with her as a member of the negotiating.\nspeaker 0: Team for the New Mile High Stadium.\nspeaker 9: She made.\nspeaker 0: Sure.\nspeaker 9: That minority participation was involved in our contracts. Denver International Airport. When we negotiated to get percentages, which of course the Supreme Court threw out, said it was unconstitutional. She supported our efforts. Hiawatha Davis, Happy Haines and I worked Debbie worked on that formative action program, goals for the airport.\nspeaker 9: Cathy was right there supporting us, giving.\nspeaker 0: Us the encouragement and negotiating those.\nspeaker 9: Those type of agreements with all the developers in this city. She never back down from anybody. She taught me how to sit down and negotiate against and 17th.\nspeaker 0: Street attorneys.\nspeaker 9: And so did the news. We sat together on the mall and on the stadium, Mile High Stadium. I never did that before. I was just a kid from the west side that grew up in the city, never went to West High School, never thought I would be sitting on city council, never becoming the president like I became\nspeaker 9: a president of the city council. But Cathy Reynolds broke that glass ceiling. Not for me, not just for me, but for all of you up there as well. So nothing is more appropriate than having this named after her.\nspeaker 9: So thank you all for doing that.\nspeaker 0: Thank you. Our next speaker this evening is Judy Gold. Thank you, Madam President. Members of the City Council. My name is Judy Gold, and I live in Washington, D.C.. I am honored to be here this evening to speak in strong support of Council Bill 21 1322, to name this City Council Chambers for Councilwoman Cathy Reynolds.\nspeaker 0: It was important for me to come all this way to speak. I worked for Cathy Reynolds for 18 years, from 1975 to 1993. I was the first council person, personal staff assistant ever hired in 3 minutes or 30 minutes.\nspeaker 0: I cannot cover all that Cathy accomplished, but I will try to highlight some of her more creative legislative moments. Denver is a greater city today because of Cathy Reynolds. Her innovative work resulted in a way to issue municipal bonds that eliminate conflicts of interest.\nspeaker 0: Psychological testing and training is now required for all police officers to ensure they are qualified to protect and serve, and women can become cops and firefighters. Cathy wrote a law increasing transparency between neighborhood groups in the city via direct lines of communications.\nspeaker 0: In 1990, she authored and passed the gay rights bill that was upheld by the U.S. Supreme Court. Because of her, you have a ban on assault weapons that was also upheld by the U.S. Supreme Court. She wrote The Denver law that allows women to seek lawful abortions without fear of harassment.\nspeaker 0: But it was her day to day work that made Cathy most proud. Whether on economic development, housing, health and hospitals, the developmentally disabled or the city budget or even trash, potholes or zoning. Cathy put her heart and mind into all issues.\nspeaker 0: She would cut a deal, but she would never sacrifice her principles. My favorite quote is from her first reelection way back in 1979, when I asked someone if you would vote for her, and he said yes, because he didn't always agree with her, but she would always give him the straight story and never change her position from\nspeaker 0: one person to the next. That was and still is the best compliment ever, because her constituents always knew where she stood, whether or not they agreed with her. Councilwoman Kathy Carter. Reynolds was a unique person and an exceptional elected official.\nspeaker 0: Her constant dedication, coupled with her wry sense of humor, gave her this rare characteristic that provided Denver with quality legislation, funding national recognition and a plan for the future. For all of these reasons and many more. It is extremely fitting that this council chambers be named for her in the future.\nspeaker 0: When people ask, Who is Kathy Reynolds? Someone can explain. She was the best this wonderful city has to offer. I ask for your support. Thank you for your time. Thank you. Our next speaker this evening is Donna. Good.\nspeaker 0: Donna. Good. Hi. I appreciate being asked to speak tonight. I'm done a good. And until ten days ago, I lived in Park Hill for the last 21 years. I moved into Councilman Black's district ten days ago, and I don't even know where I am.\nspeaker 0: I got nosebleeds when I used to go above Alameda. So I think the reason I'm here tonight is to talk about something that you do hear about sometimes. But maybe we need to remember Cathy as somebody who accomplished everything we've heard about tonight.\nspeaker 0: And maybe we need to remember for something that we've only heard touched on a couple of times. So I'm with commonly known as Political Hack. I've run 17 elections. I worked for two presidents, two mayors, two governors, and a bunch of other people in political office.\nspeaker 0: And what I have learned in all those years is that you can spend time with people, you can work for them. And. They don't really care whether they're working for the people of a city. The people of a country, the people of a state.\nspeaker 0: Or a major corporation. They don't. But what people who are really the ones who make the difference and I can recognize them right off because I've worked with so many of both kinds. But it's what Kathy stood for.\nspeaker 0: I think L.B. mentioned it. Maybe Rosemary touched on it. Tony Ogden. Certainly. Sorry. Kathy had a passion for service. Kathy was the real deal. She cared about every one of us in Denver. And when people say she loved the city, I love Denver.\nspeaker 0: And it means I love my neighborhood, I love my corner cafe. It means I love where I walk my dog. And no matter whether I was working in Pakistan or D.C., I kept the home in Denver, and Denver was home.\nspeaker 0: Kathy loved not just the city, because what does that mean? What it means, I think, is that she loved us. She loved the people of Denver. She wanted us to have better health services. The ability to live in love as we stood.\nspeaker 0: She wanted people who had never been given a chance to be given a chance. She wanted us to be able to take our family to different venues. She loved us. And maybe it's too late to say to her face, but we can say to her family and we can remind people forever by putting in this room that\nspeaker 0: a woman worked here once who spent 28, almost 30 years her life every day dedicated to making this city a better place for each and every person who lived here. She was an early comer with things like LGBT rights.\nspeaker 0: She fought for Denver Hill. She was the real deal. And I appreciate what you're all going to do tonight. Thank you. Our next speaker this evening is Tisha. He shook.\nspeaker 9: Oh, goodness.\nspeaker 0: Hi. I'm Tisha. I live in Council District 11. Hi. So I met Cathy with my small circle of friends. We're trying to get Denver to pass an equal protection ordinance in 1988. We were young and we were dedicated and we were as ignorant as all get out.\nspeaker 0: Fortunately, there were wiser heads who decided to help us, and Kathy was one of those, as were most of the folks speaking tonight. Her support was crucial in the 1990 Council vote.\nspeaker 9: In Oliver and I will just be forever grateful to.\nspeaker 0: Her for all that.\nspeaker 9: She did. To get that past and to stay with us every time it was fought over and over. Like many. She taught me how to count to seven. What was more important to me was that lesson, and it was a continuing lesson in how to work well with others.\nspeaker 9: Especially those you disagree with. How wonderful.\nspeaker 0: To name Council Chambers.\nspeaker 9: For her. I really hope.\nspeaker 0: That her name and her legacy serve to inspire future council members. Thank you, Councilman Flynn and Councilwoman Ortega for doing this. Thank you. Thank you. Our next speaker is Joyce Foster. And then we're going to follow with Martin Foley.\nspeaker 0: I've. Good evening, ladies and gentlemen of the Denver City Council. It is truly a pleasure being in front of you this evening. This was my happiest job. I ran for election when Stephanie Foote was appointed by Mayor Webb to become his chief of staff.\nspeaker 0: And I was recruited. To run. And the only time I ever had been down here was during the amendment that Kathy was running for. Well, if it wasn't for you. My husband was very involved with her. He was the co-chair of No.\nspeaker 0: One to campaign. So I was elected as the first Jewish woman to sit on the Denver City Council and bring that community with me. And I had no idea how many people sat on the city council when I entered the race.\nspeaker 0: I was very naive. I had been working for Jewish family service for 17 years and my whole life I had been recruited. I had been resettling immigrants from the former Soviet Union, from Laos boat people and helping them find jobs and homes.\nspeaker 0: That's what I did for a living. And after I was elected, my first job on city council was to break a tie on who was to be elected city council president. It was between Cathy Reynolds, who I knew a little bit from the the work that she did with my husband and another gentleman who will go nameless\nspeaker 0: . I really thought, well, she's been here so long. She's been president of city council. I'll add somebody else. I'll vote for somebody else. So I voted for the other guy. Whoa. Was that a bad move on my part?\nspeaker 0: Yeah. I was soon to lose my best friend, and I apologized about a year later after I saw and was learning. You know what she did and how well she did it? And I did apologize. And I said, whenever you run for city council president again, I promise I'm going to support you because you are really smart\nspeaker 0: . You're thoughtful. You're respectful. And she's really, really smart. Really smart. And I loved Kathy. She became such a good friend. And Steven and I miss her. We miss Rick. We miss our community. I was on the council for ten years until I was term limited and.\nspeaker 0: Sometimes we would get together and just visit and it was wonderful. And I miss Kathy and I can't imagine a better name for this cherished hall, this treasure that we have in Colorado. So I definitely support this bill.\nspeaker 0: Thank you. Or whatever it is. It's. Thank you. Our next speaker is Martin Place.\nspeaker 9: Members of council. Hi, my name is modestly. I'm a Capitol Hill resident for about 50 years. But probably the longer stake is that I was a city employee for 37 years. Most of that was as a career employee.\nspeaker 9: I was an appointee for under Mayor Webb, and then I was a contract employee. So I sort of every flavor of employee you can have. And I go back to when Kevin Flynn was one of those annoying reporters in City Hall and Debbie was a legislative aide, if I remember correctly, when we first met.\nspeaker 9: So those are the familiar faces on council. Kevin, I should have asked you, was this a promotion or a demotion going from reporter this. So I started five years before Cathy Reynolds became a member of council, had occasion to work with her on a whole host of other of issues because I was the face of the finance\nspeaker 9: office for a much of that time. The main thing I want to affirm is how the extraordinary leadership and sense of service that Cathy demonstrated, manifested and that we others were able to learn from. I knew it. The mayors knew it.\nspeaker 9: The council members knew it. And the people knew it. If you wanted to find out what the straight story is, you talk to Cathy Reynolds. She was also a friend. And I'm very grateful that you're recognizing her in this way.\nspeaker 9: Thank you.\nspeaker 0: Thank you. We wanted to see if we might have Wellington Webb in the Zoom platform. All right. Okay. Well, we tried. We will go ahead and move forward then. That concludes our speakers this evening. Thank you, everyone, for sharing your memories and perspective of her and questions from members of Council on Council.\nspeaker 0: Bill. One, three, two, two. All right. Seen no questions by members of council. The public hearing is closed. Comments by members of Council on Council Bill one, three, two, two. And I hope that we might start out with council members Flynn and Ortega, and we can follow with other members of council after they go.\nspeaker 6: Certainly. Thank you, Madam President. I don't want to lengthen the mindful of what Tim Sanders had said. I want to have a note pass down from Council President Gilmore, but I just want to thank everybody for showing up and reminding folks of what public service is all about, what change is all about, and what meeting the challenge\nspeaker 6: of change is all about, and what it takes to do that. And a number of the people in this room who worked with Cathy in accomplishing that. Really tell that story. It was a group effort. It took a leader, but it took the whole village.\nspeaker 6: Really? Ramona Councilwoman Martinez, your words really, really struck me. And still to this day, your story of how you grew up here and then became yourself president of the council and with the help of Councilwoman Reynolds mentorship. When you worked for Sam and then later when you joined her as a as an equal colleague.\nspeaker 6: Every time I come to this building, I drive by the house that you told me you grew up in, that tiny little bungalow. I can imagine that the little girl who grew up in that rose to the accomplishments that you did and having broken, having benefited from the glass ceiling that that Kathy Reynolds broke.\nspeaker 6: And I and I would have similar remarks for everyone else in the room for for former Councilwoman Haynes, for Councilwoman Foster. Councilwoman Foote. Am I missing anybody? Councilwoman Marcia Johnson was your. Councilman Santos this I can't imagine a better way to memorialize the record of accomplishment that she had than in naming this room.\nspeaker 6: As I said at the end of the presentation, this room where it happened, it's the appropriate place to do it. We have memorialized other people by naming elements of buildings. John Parr, Sandy Weidner, we've recognized with the PA Weidner room, and I think this is a crowning honor for a career that really changed Denver.\nspeaker 6: And with that, Madam President, I would certainly ask for everyone's support for this bill. Thank you.\nspeaker 0: Thank you. Councilmember Flynn. Councilmember Ortega. Thank you, Madam President. So I had the pleasure of being a council aide along with Judy Gold, along with Ramona martinez, Happy Haines. During the time that she had first got elected in her first term, and we all went on to accept, Judy, to replace the council members that we served with\nspeaker 0: , she was an inspiration to all of us to want to to be like her, to want to make a difference in our community, and to step into those big shoes that were filled by our predecessors. And, you know, to see that we still have this many women serving on the council.\nspeaker 0: She truly did break that glass ceiling for for all of us. And she was she was a dear friend. We all loved her. I mean, just the fact that this many of her former colleagues are here tonight to express those words of appreciation and love that we all had for her.\nspeaker 0: I just thank you all for being here tonight. And I really would appreciate a unanimous support of this bill tonight so that we could officially change the name of this chamber after one of our great leaders, Kathy Reynolds.\nspeaker 0: Thank you. Thank you, Councilmember Ortega. Councilmember Kinney.\nspeaker 8: Thank you. Council President. I am part of the next generation right of council members. So unlike some of the folks who are here tonight, I didn't serve as a council aide and I was a young mom a and looking to run for office as the first out member of the council.\nspeaker 8: We don't know. There may have been other LGBTQ members, but the first one to kind of run as an out individual and Maria Garcia Berry made the introduction and said, you need to meet Cathy. And not just because you're running for the At-Large seat, because you and Cathy, Cathy, Cathy, you know, had a brand of leadership that\nspeaker 8: she she thought I needed to understand and know about. And it was pretty intimidating, actually, to learn about her history in the office and, you know, to be someone who didn't come from inside the city family, because that was clearly something that was really important to a lot of Cathy and her predecessors.\nspeaker 8: And so but I also was really inspired. It struck me the first time I met her, like I might not even have the opportunity to run for an office like this with the background I have, if it weren't for Cathy.\nspeaker 8: And so I was so nervous too to try to live up to this idea that and people would ask me, they'd say, you know, what do you want to do as the first out lesbian in city council? Like what big policies are you going to run for the community?\nspeaker 8: And I would say. The important thing to understand is that straight allies paved the way. That it took those from communities who had not lived in the shoes of being a gay person to pass laws that were inclusive.\nspeaker 8: And that was really important to me, thinking about being a leader, right? When I worked on immigrant issues, I have not walked the shoes of an immigrant in our city or in our country. But having to think about I can't wait for for for only people with that background to lead or, you know, police accountability where, you\nspeaker 8: know, we have a huge community of individuals of color and men of color in particular, to whom police accountability is really important. I can't only represent that issue if I've walked in their shoes. Cathy inspired me and sent the message that you have to lead for all of those communities with their experience and partnership, right?\nspeaker 8: Hearing TE and others talk about how she worked with you and that model of trying to see, I might not have walked in those shoes, but if I can be alongside you, right? Those are the things that I took.\nspeaker 8: And in my early terms on the council, I would get the short emails. You know, some of you might have served before the email era, but I would get the short email from Cathy and I would pretty much know in that one sentence if she liked or did not like a quote in the paper or a vote\nspeaker 8: that I had taken. And I learned to pick up the phone and call her, especially when she didn't like it and she was so curious. She would always actually want to hear why I had voted differently than she thought.\nspeaker 8: And she never cut me off, right? She never was like, You're dead to me because you voted against. She would say, okay, well, I hate it, but you know, and then we'd move on and I'd get the next email and I had done okay.\nspeaker 8: She was still serving on the Citizens Oversight Board in my first term, and that was a topic that we talked about right when there were some hard moments. Our our former monitor had a little more contentious relationship with our safety agencies than our most recent monitor did.\nspeaker 8: And she, as someone who loved the city, loved the mayor and loved the safety agencies, was also really willing to hold them accountable. She did not think those two things were mutually exclusive. And I continue to feel that same way that I can love the city while also wanting us to do better and holding us accountable to\nspeaker 8: those high standards. So those are the things I've taken personally. And as her health began to struggle, we tried to remember her in these chambers in terms of celebrating the anniversaries of the nondiscrimination ordinance as part of the history.\nspeaker 8: And I remember going to visit her at the house. And I will just say for me personally, I often heard from Rick and Cathy together. I kind of feel like I got to know you together. And it's for me very fitting that so much of her service you are so involved in.\nspeaker 8: And so I want to say thank you to you as well, because that inspiration, the trust, the watchful eye, all of that, I feel like was really something you did as a team, and it meant a lot to me, even though I did not work with her in the same way you all did.\nspeaker 8: I felt that mentorship and I felt that support and I felt those high standards and they made me a better councilwoman. So thank you to all of you for the service that you performed for our city and thank you to her family.\nspeaker 8: We know a little bit about the shared sacrifice that a family has for service. And so I thank you for the last dinners, the interrupted birthdays and all the other things that helped make her service to our city so possible.\nspeaker 8: So I'm really excited and thank you to my colleagues for bringing this forward. I think you found the perfect place. So thank you.\nspeaker 0: Thank you. Councilmember Kenney, Councilman Cashman.\nspeaker 3: Know. Thank you, Madam President. I thank you, Councilwoman Ortega, Councilman Flynn, for bringing this this forward. This this pretty humbling for me. I've been I grew up in the Garden State of New Jersey. And on Halloween night, just a couple of weeks ago, I celebrated my 50, 50 years in Colorado, all of that, except for maybe one\nspeaker 3: year in the city and county of Denver. And for a long time as a journalist, I had a community newspaper in South Denver. I spoke to a number of the folks in the room here, including Councilwoman Reynolds. But what I'm remembering when I when I was thinking about running and was even 13 or 14, I called my\nspeaker 3: friend Lynn Preston Hall, who was an aide for Councilwoman Johnson. And I said, you know, how much time does this job take? And she said, Well, you know, she said, Paul, it would be up to you. You know, some people talk about working and then others of them actually work.\nspeaker 3: And she said, My boss works so. I took that to heart and I've tried to work. I don't know that I would ever even aspire to the heights that Ms.. Reynolds achieved, but I do try to try to work.\nspeaker 3: You know, I did speak with Councilwoman Reynolds a few times as a journalist, and I found her to be as described by some of the speakers. She was always would return my call, but she did not seem to be a particular fan of talking to the press, and she was always very direct.\nspeaker 3: She would explain fully, answer my questions fully, but she was not a look at me person. She was. My sense was she was a let's let's get it done type of person. And as I said, you know, she was very direct and to the point.\nspeaker 3: But I did she had the opportunity to see another side of her. The sense of humor that people talk about. I happened to be as actually to I believe that were Fridays to Fridays in a row. I happened to be playing golf over on a golf course and came off the first green.\nspeaker 3: And there was this woman laughing her tail off on the second tee about 20 yards away and got a little bit closer. And it was Councilwoman Reynolds and I don't know what the topic was, but she was laughing about 200 yards down the fairway and having a great time.\nspeaker 3: So I was pleased to hear that side of her. And the other thing I wanted to comment about when I first got elected to office, Albert Brooks was president at the time of council, and he appointed me. To the Urban Drainage and Flood Control Board, and who knew what that could possibly be?\nspeaker 3: And why would I have been subjected to this? And what I came to find out, first of all, is it is one of the most well-run, marvelous organizations I've ever had the privilege to be associated with. I want to acknowledge Dave Bennetts, who's I think government relations manager now for the district and has had a remarkable career\nspeaker 3: for that organization. And the other thing I want to say is when I took the position and Councilwoman Sussman was just leaving that role on the board, the only thing she had to tell me, she came over and whispered.\nspeaker 3: She said, you had the best lunches you're ever going to eat. So, yeah, they they do a good job in that way. But I do want to thank all all the former council members here, former attorney muse Mr. Reynolds, and all the family members for for what you did in setting the table so all of us can\nspeaker 3: be up here working for a city that's with all of its challenges in pretty good shape. So thank you, Madam President.\nspeaker 0: Thank you, Councilmember Cashman. Councilmember Hines.\nspeaker 3: Thank you. Council President, I this is my first term in office and I never met Miss Reynolds. Did not know her. I. Would say tonight, I want to thank the sponsors for their fabulous presentation. And and I want to thank the folks here in this room.\nspeaker 3: It's just been such a delight to learn more about Ms.. Rental, Ms.. REYNOLDS And what a force she has been in our city. And I am also speaking of delight. I am delighted that she broke a barrier and helped us become, as you have already said, such a diverse council with majority women council, too.\nspeaker 3: And so I just wanted to thank everyone for coming. I feel a little bit I feel like I know her a little bit better, and I'm excited to have this chambers named after her. Thank you.\nspeaker 0: Thank you. Councilmember Hines. Councilmember Sandoval.\nspeaker 5: Thank you, Madam President. Um, so I never had the honor of knowing Cathy, but I know my father, Paul Sandoval, did, and worked with her a great deal. And as a daughter of someone who was elected, I had to share my dad with the community.\nspeaker 5: And I remember going places with my dad, and I would get tired of him talking to everyone. And I would just want my dad's attention. And we would go to on a cruise. I'll never forget we were in Mexico and my dad knew someone on the cruise in Mexico.\nspeaker 5: And it all revolved around Denver politics. And so as a woman who is now currently elected and I have two children at home, I know the strain my dad carried and the strain that she probably carried of being a mother and being a wife and being a politician.\nspeaker 5: And now I see my kids. I go places and my kids are like, Mom, come on, let's go. Do we have to keep talking? And so I just wanted to thank her family, her husband, who sat by her side.\nspeaker 5: My husband's been there for me. I don't think anyone really knows what that's like. And her kids who she had, you had to share your mom with the community and what that feels like of people telling you stories about your mom that you didn't know and exploring things and hearing things.\nspeaker 5: When I was campaigning, I learned more about my dad than I had known. I didn't know. I knew he had a big impact in northwest Denver, but I really learned it. Going knocking door to door and hearing. So just wanted to thank her family because this is a job where you don't get to go home when you\nspeaker 5: just turn it off. You don't. You're everywhere you go. You're Councilwoman Sandoval. You're not the story. And so thank you for her family, for standing by her side for so long and standing in her, because I'm sure she needed to come home and she rented to you all, just like we all as mothers up here and fathers\nspeaker 5: go home. And we went to our significant other in our kids. And so we're all in it together. And for all of you, other women who were elected, thank you for opening the door for me and for my family and for my ability to serve Northwest Denver and get things done.\nspeaker 5: Thank you.\nspeaker 0: Thank you, Councilmember Sandoval and see no other speakers in the queue. I'll go ahead and round it out tonight. I want to sincerely thank council members Flynn and Ortega for sponsoring this. And I see when Council member Flynn started to try to go down the roll call, it makes it tough.\nspeaker 0: So I would just like to acknowledge and sincerely and humbly thank the former council members who are here with us this evening as well and online. You know, some of the things that were said about Kathy Reynolds tonight were she was a strong woman.\nspeaker 0: She was a visionary. She was a passionate servant. She was a get it done sort of person. Her humor, her patience, her kindness and her generosity. Were things that you said over and over again in a little bit different way, but it gave us an opportunity sitting up here tonight to fill.\nspeaker 0: For the ones of us that never met her, feel like we knew her just a little bit. And the humility and. Just great honor that it is to sit up here and serve our city and know the intentional work that it does take to do this job to the best of your ability and to her husband and\nspeaker 0: her two children. I know a couple of my colleagues brought it up, but your sacrifices, because I think that she couldn't serve for almost 30 years and there weren't sacrifices that you all made as family. And that sacrifice changed policy and legislation in Denver for the betterment of citizens that you will never meet.\nspeaker 0: But she did that and you allowed her to do that because having the support of your family members, when you go home to not talk about what happened, to talk about what happened in your life and in your day is so refreshing and so important for us to be able to continue doing our jobs and the coalition\nspeaker 0: building. How she served on the the Citizen Oversight Board. Women's rights. I have two daughters and I want them to consider public service at some point. I want them to know that it's achievable for them and that they can do that and have a city or have excuse me, have a family and still serve the city in\nspeaker 0: a good way, but that you're also serving your family at the same time. And so I'm very honored to support this this evening. And, Madam Secretary, roll call, please. Flynn.\nspeaker 6: Just checking what the snow says. I don't have seven votes. I.\nspeaker 0: Ortega, i. And we would like to ask council members who are in the chambers to all please stay so we can take a group photo with you and our seated council.\nspeaker 7: I think Sandoval. Sawyer.\nspeaker 0: I. Torres, I. Black. I.\nspeaker 7: See tobacco.\nspeaker 5: I. Clark.\nspeaker 6: I.\nspeaker 0: Herndon.\nspeaker 1: I.\nspeaker 3: Times i.\nspeaker 7: Cashman.\nspeaker 0: Can I?\nspeaker 7: Madam President.\nspeaker 0: I. Madam Secretary, closed the voting and announced the results.\nspeaker 7: 13 eyes.\nspeaker 0: 13. I count the bill. 21 guy. 13. 20. They're being for no further business before this body. This meeting is adjourned. Thank you.\n\n"

In [15]:
def file_inference_mb(question, material):
    prompt = prompt_no_input.format_map({"instruction": material + "\n%s"%question})
    output = respond(prompt=prompt)
    print(output)

In [16]:
file_inference_mb(question, material)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


OutOfMemoryError: CUDA out of memory. Tried to allocate 152.00 MiB. GPU 0 has a total capacty of 22.19 GiB of which 19.50 MiB is free. Including non-PyTorch memory, this process has 22.16 GiB memory in use. Of the allocated memory 18.54 GiB is allocated by PyTorch, and 3.32 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### File

In [ ]:
def file_inference_mb(question, in_file, out_file):
    with open(in_file, "r") as f:
        data = json.load(f)
    with open(out_file, "w") as f:
        for meeting in tqdm(data):
            sample_result = {}
            material = meeting["source"]
            target = meeting["summary"]
            prompt = prompt_no_input.format_map({"instruction": material + "\n%s"%question})
            output = respond(prompt=prompt)
            sample_result["target"] = re.sub(r'\n', '', target)
            sample_result["prediction"] = re.sub(r'\n', '', output)
            json.dump(sample_result, f)
            f.write('\n')

In [ ]:
in_file = "../../MeetingBank/test_text.json"
out_file = "./output/output_MB_text.json"

In [16]:
file_inference_mb(question, in_file, out_file)

  0%|          | 0/159 [00:00<?, ?it/s]

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
  1%|          | 1/159 [00:31<1:23:40, 31.78s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 134.00 MiB. GPU 0 has a total capacty of 22.19 GiB of which 9.50 MiB is free. Including non-PyTorch memory, this process has 22.17 GiB memory in use. Of the allocated memory 18.19 GiB is allocated by PyTorch, and 3.67 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF